<a href="https://colab.research.google.com/github/ayush-dhanker/Image-Classification-on-MNIST/blob/main/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as py

In [ ]:
# normalizing and transforming
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

# Loading Datasets
train_dataset=datasets.MNIST(root='./data',train=True,download=True,transform=transform)
test_dataset=datasets.MNIST(root='./data',train=False,download=True,transform=transform)

# Data loaders
train_loader=DataLoader(train_dataset,batch_size=128,shuffle=True, drop_last=True, num_workers=5)
test_loader=DataLoader(test_dataset,batch_size=128,shuffle=False, num_workers=8)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# building model
model= nn.Sequential(
      nn.flatten(),
      nn.linear(784,512),
      nn.ReLU(),
      nn.linear(512,128),
      nn.ReLU(),
      nn.linear(128,10).to(device)
)

print("model = ",model)
with torch.no_grad():
    print("Maximum weight before custom init: ", model[1].weight.max())


def glorot_init(layer: nn.Module):
    if isinstance(layer, nn.Linear):
        nn.init.xavier_uniform_(layer.weight)
        nn.init.zeros_(layer.bias)


with torch.no_grad():
    model.apply(glorot_init)
    print("Maximum weight after custom init", model[1].weight.max())

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=0.03)

In [ ]:
# train_model
def train_model(
    model: nn.Module,
    loss_fn: nn.Module,
    optimizer: optim.Optimizer,
    training_loader: DataLoader,
    validation_loader: DataLoader,
    n_epochs:int,
    verbose:bool=True
    ):

  train_lem=len(training_loader.dataset)
  steps_per_eppoch=train_len/training_loader.batch_size

  print("Running {} at {} steps per epoch ").format(n_epochs, steps_per_epoch)

  train_acc=[]
  train_loss=[]
  val_acc=[]
  val_loss=[]

  for epoch in range(n_epochs):
    epoch_train_loss=0
    epoch_train_acc=0





  def training_set(
      input: torch.tensor,
      label: torch.tensor,
      model: nn.Module,
      loss_fn: nn.Module,
      optimizer: optim.Optimizer):

    input=input.to(device)
    label=label.to(device)
    output_batch=model(input)
    loss_batch=loss_fn(ouput_batch, label)

    loss_batch.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
      batch_acc=accuracy(output_batch,label)
    # see loss_batch.item()
    return loss_batch,batch_acc



  def evaluate(
      model:nn.Module,
      dataloader:DataLoader,
      loss_fn:nn.Module):
    model.eval()
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    loss=0
    correct=0

    with torch.no_grad():
      for input,label in dataloader:
        input=input.to(device)
        label=label.to(device)
        prediction=model(input)
        val_loss=loss_fn(prediction,label)
        correct += (prediction.armax(axis=1)==label).type(torch.float).sum().item()

    loss/=num_batches
    val_accuracy=correct/size
    return loss, val_accuracy

  def accuracy(labels:torch.tensor,
               outputs:torch.tensor)=>torch.tensor:
               predictions=torch.argmac(outputs,axis=-1)
               matches=labels==predictions
               return matches.float().mean()




In [ ]:
metrics = train_model(model, loss_fn, optimizer, train_dataloader, test_dataloader, n_epochs=25)